In [1]:
!pip install pillow tqdm

In [59]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from PIL import Image
from multiprocessing import Pool
from tqdm import tqdm

# Dataset paths — update if using Google Drive
xml_root = "/content/drive/MyDrive/driving-video-with-object-tracking/IDD/extracted/IDD_Detection/Annotations"
img_root = "/content/drive/MyDrive/driving-video-with-object-tracking/IDD/extracted/IDD_Detection/JPEGImages/"
out_dir = "/content/drive/MyDrive/driving-video-with-object-tracking/bdd100k_yolov11n"

# Split & workers
train_ratio = 0.6
num_workers = 8


In [2]:
# Class mappings
class_map = {
    'bicycle': 0, 'bus': 1, 'car': 2, 'motorcycle': 3,
    'other person': 4, 'other vehicle': 5, 'pedestrian': 6,
    'rider': 7, 'trailer': 8, 'train': 9, 'truck': 10,
    'animal': 11, 'autorickshaw': 12, 'caravan': 13,
    'traffic light': 14, 'traffic sign': 15
}

remap = {
    'person': 'pedestrian',
    'vehicle fallback': 'other vehicle'
}


In [46]:
for split in ['train', 'test']:
    os.makedirs(f"{out_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{out_dir}/labels/{split}", exist_ok=True)


In [20]:
def get_all_files(base_dir, ext):
    return [os.path.join(r, f)
            for r, _, files in os.walk(base_dir)
            for f in files if f.endswith(ext)]

# Collect all .xml and .jpg files
xml_paths = get_all_files(xml_root, '.xml')

# Create pairs that preserve folder hierarchy
paired_paths = []
for xml_path in xml_paths:
    rel_path = os.path.relpath(xml_path, xml_root).replace('.xml', '.jpg')
    img_path = os.path.join(img_root, rel_path)
    if os.path.exists(img_path):
        paired_paths.append((xml_path, img_path, rel_path))  # Add relative name for output folder

In [55]:
random.shuffle(paired_paths)
split_idx = int(len(paired_paths) * train_ratio)
stop_idx = split_idx + int(0.15 * len(paired_paths))
train_pairs = paired_paths[:split_idx]
test_pairs = paired_paths[split_idx: stop_idx]


In [56]:
print(len(train_pairs), len(test_pairs))

25114 6278


In [23]:
print(len(xml_paths))

41857


In [24]:
print(len(img_paths))

46659


In [25]:
#print unique img_paths
print(len(set(img_paths)))

46659


In [26]:
print(len(train_xmls), len(test_xmls))

418 100


In [27]:
def convert_xml_to_yolo(xml_path, label_out_path, img_size):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        w, h = img_size

        lines = []
        for obj in root.findall('object'):
            cls_orig = obj.find('name').text.strip()
            cls = remap.get(cls_orig, cls_orig)
            if cls not in class_map:
                continue
            cls_id = class_map[cls]

            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / w
            y_center = ((ymin + ymax) / 2) / h
            box_w = (xmax - xmin) / w
            box_h = (ymax - ymin) / h

            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

        os.makedirs(os.path.dirname(label_out_path), exist_ok=True)
        with open(label_out_path, 'w') as f:
            f.write('\n'.join(lines))
        return True
    except:
        return False


In [35]:
def process_file(xml_path, img_path, rel_name, split):
    folder_tag = os.path.dirname(rel_name).replace(os.sep, '_')
    base_name = os.path.basename(rel_name)
    new_name = f"idd_{folder_tag}_{base_name}"

    img_out = os.path.join(out_dir, f"images/{split}", new_name)
    lbl_out = os.path.join(out_dir, f"labels/{split}", new_name.replace('.jpg', '.txt'))

    try:
        shutil.copyfile(img_path, img_out)
        with Image.open(img_path) as img:
            img_size = img.size
        return convert_xml_to_yolo(xml_path, lbl_out, img_size)
    except:
        return False


In [36]:
def run_parallel(pairs, split):
    print(f"🔧 Processing {split} split...")
    args = [(xml, img, rel, split) for xml, img, rel in pairs]
    with Pool(num_workers) as pool:
        results = list(tqdm(pool.starmap(process_file, args), total=len(args)))
    print(f"✅ Completed {sum(results)} / {len(results)}")


In [49]:
#safely remove existing files in /content/first_run
#shutil.rmtree(out_dir, ignore_errors=True)

In [60]:
run_parallel(train_pairs, "train")

run_parallel(test_pairs, "test")

🔧 Processing train split...


100%|██████████| 25114/25114 [00:00<00:00, 5421294.42it/s]

✅ Completed 25114 / 25114
🔧 Processing test split...



100%|██████████| 6278/6278 [00:00<00:00, 4947734.03it/s]

✅ Completed 6278 / 6278


In [58]:
#count the number of train and test images and labels when BDD samples are present.
train_images = len(os.listdir(os.path.join(out_dir, "images/train")))
train_labels = len(os.listdir(os.path.join(out_dir, "labels/train")))

test_images = len(os.listdir(os.path.join(out_dir, "images/test")))
test_labels = len(os.listdir(os.path.join(out_dir, "labels/test")))

print(f"Train images: {train_images}")
print(f"Train labels: {train_labels}")
print(f"Test images: {test_images}")
print(f"Test labels: {test_labels}")

Train images: 39063
Train labels: 37839
Test images: 19880
Test labels: 19175


In [1]:
#count the number of train and test images and labels after merging IDD data samples
train_images_final = len(os.listdir(os.path.join(out_dir, "images/train")))
train_labels_final = len(os.listdir(os.path.join(out_dir, "labels/train")))

test_images_final = len(os.listdir(os.path.join(out_dir, "images/test")))
test_labels_final = len(os.listdir(os.path.join(out_dir, "labels/test")))

print(f"Train images: {train_images_final}")
print(f"Train labels: {train_labels_final}")
print(f"Test images: {test_images_final}")
print(f"Test labels: {test_labels_final}")

NameError: name 'test_labels_final' is not defined

In [16]:
from types import SimpleNamespace
from yolox.tracker.byte_tracker import BYTETracker

tracker_args = SimpleNamespace(
    track_thresh=0.5,
    match_thresh=0.8,
    track_buffer=30,
    frame_rate=30,
    min_box_area=100
)

tracker = BYTETracker(tracker_args, frame_rate=tracker_args.frame_rate)

ModuleNotFoundError: No module named 'yolox.tracker'

In [7]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
!sed -i 's/onnx-simplifier==0.4.10/onnx-simplifier>=0.4.33/' requirements.txt
!pip install -r requirements.txt
!pip install -v -e .

Cloning into 'YOLOX'...
remote: Enumerating objects: 1940, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 1940 (delta 5), reused 1 (delta 1), pack-reused 1924 (from 3)
Receiving objects: 100% (1940/1940), 7.56 MiB | 33.06 MiB/s, done.
Resolving deltas: 100% (1152/1152), done.
/content/YOLOX/YOLOX
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
  Using cached onnx-1.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 137.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.

In [9]:
!git clone https://github.com/ifzhang/ByteTrack.git
import sys


Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Counting objects: 100% (668/668), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 2007 (delta 512), reused 491 (delta 491), pack-reused 1339 (from 1)
Receiving objects: 100% (2007/2007), 79.58 MiB | 50.74 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.


In [14]:
sys.path.append('/content/YOLOX/YOLOX/ByteTrack')

In [13]:
!find /content -name "byte_tracker.py"

/content/YOLOX/YOLOX/ByteTrack/tutorials/qdtrack/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/centertrack/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/jde/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/transtrack/mot_online/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/cstrack/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/motr/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/trades/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/ctracker/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/tutorials/fairmot/byte_tracker.py
/content/YOLOX/YOLOX/ByteTrack/yolox/tracker/byte_tracker.py


KeyboardInterrupt: 

In [17]:
import importlib.util
spec = importlib.util.spec_from_file_location("byte_tracker", "/content/YOLOX/YOLOX/ByteTrack/tutorials/fairmot/byte_tracker.py")
byte_tracker = importlib.util.module_from_spec(spec)
spec.loader.exec_module(byte_tracker)
BYTETracker = byte_tracker.BYTETracker

from types import SimpleNamespace

tracker_args = SimpleNamespace(
    track_thresh=0.5,
    match_thresh=0.8,
    track_buffer=30,
    frame_rate=30,
    min_box_area=100
)

tracker = BYTETracker(tracker_args, frame_rate=tracker_args.frame_rate)
print("Successfully imported and initialized BYTETracker")

ModuleNotFoundError: No module named 'models.model'

In [18]:
!ls -l /content/YOLOX/YOLOX/ByteTrack/tutorials/fairmot/

total 44
-rw-r--r-- 1 root root 14843 Jul 27 04:51 byte_tracker.py
drwxr-xr-x 2 root root  4096 Jul 27 04:59 __pycache__
-rw-r--r-- 1 root root   501 Jul 27 04:51 README.md
-rw-r--r-- 1 root root 17585 Jul 27 04:51 tracker.py


In [20]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00


In [22]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [24]:
model = YOLO("/content/best.pt")

In [25]:
print(model.names)


{0: 'bicycle', 1: 'bus', 2: 'car', 3: 'motorcycle', 4: 'other person', 5: 'other vehicle', 6: 'pedestrian', 7: 'rider', 8: 'trailer', 9: 'train', 10: 'truck'}
